### To Run
- Have access to TCNSIntern ID (It uses content from drive)
- Clone repository (Code in cell below. Just run)
- Drag `colorlist` folder from the repo folder to Colab's runtime (content/)
- Drag `seg_import.py` and `extractor.py` from repo filder to Colab's runtime (content/)
- In `extractor.py` line 13, change the path of `colorlist.txt` to current path `/content/colorlist/colorlist.txt`
- Upload input image to Colab's runtime (content/)
- Only run main cell after running the requirement cells

## Requirement Cells

In [1]:
!python --version

Python 3.8.10


In [ ]:
!pip install ultralytics==8.0.28

In [ ]:
!pip install extcolors webcolors

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


###SKIP

In [2]:
#Model support file - seg_import.py

from ultralytics import YOLO
import numpy as np

class segment:
  def __inti__(self, model_path):
    self.model = YOLO(model_path)

  def detect(self, img):

        h, w, c = img.shape

        #Model prediction
        results = self.model.predict(source = img.copy(), save = False, save_txt = False)
        result = results[0]

        seg_contours_id = []

        for seg in result.masks.segments:

            seg[:, 0] *= w
            seg[:, 1] *= h
            segment = np.array(seg, dtype = np.int32)
            seg_contours_id.append(segment)

        #Bboxes are the bounding boxes points ((x1,y1),(x2,y2)) and bboxes1 are the xy values with dimensions (width, heigth)
        bboxes = np.array(result.boxes.xyxy.cpu(), dtype = 'int')
        bboxes1 = np.array(result.boxes.xywh.cpu(), dtype = 'int')
        score = np.array(result.boxes.conf.cpu(), dtype = 'float')

        return bboxes, bboxes1, seg_contours_id, score 

In [8]:
#Color extraction support file - extractor.py

from scipy.spatial import KDTree
import webcolors
import extcolors

class extract:
    def __init__(self) -> None:
        pass

    def extract_color(self, img):
        colors, pixels = extcolors.extract_from_image(img, tolerance = 10, limit = 3)
        
        #Read color names from colorlist
        color_names = []
        with open(r'/content/Color-Co-ordinate-Mapping/colorlist/colorlist.txt','r') as fp:
            for line in fp:
                x = line[:-1]

                color_names.append(x)
        #RGB values for the colors in the colorlist
        rgb_vals = [(255,87,51),(248,131,121),(255,255,255),(128,0,0),(255,219,88),
                    (230,232,250),(128,128,128),(255,255,0),(194,178,128),(0,255,0),
                    (221,160,221),(128,0,128),(255,215,0),(250,249,246),(255,0,0),
                    (0,0,128),(0,0,0),(255,192,203),(128,128,0),(224,176,255),
                    (165,42,42),(255,253,208),(192,192,192),(255,229,180),(245,245,220),
                    (0,128,128),(255,165,0),(0,0,255),(184,115,51)]
        kdtree = KDTree(rgb_vals)

        extract_colors = []

        #Adds the colors whose RGB values are closest to the detected colors
        for color in colors:

            rgb = color[0]

            closest_color_idx = kdtree.query(rgb)[1]
            closest_color = color_names[closest_color_idx]
            extract_colors.append(closest_color)

        if len(extract_colors)>2:
            return extract_colors[-2:]
        else:
            return extract_colors

In [5]:
!git clone https://github.com/Adarsh-gif-crypt/Color-Co-ordinate-Mapping

Cloning into 'Color-Co-ordinate-Mapping'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 51 (delta 1), reused 8 (delta 1), pack-reused 43
Unpacking objects: 100% (51/51), 46.40 MiB | 8.37 MiB/s, done.


## MAIN CELL

In [10]:
#File which initiates the whole process - main.py

import cv2
from seg_import import segment 
from PIL import Image
import numpy as np
from extractor import extract 

#Initialize both the extractor and model 
#Enter the model weight path below
seg = segment('/content/gdrive/MyDrive/weights/best.pt')
ext = extract()

#Enter the input image path below
img = cv2.imread('/content/inp5.jpg')

#Mask
mask = np.zeros_like(img)

#Detection of the bottomwear
bboxes, bboxdim, segmentations, scores = seg.detect(img)

cv2.fillPoly(mask, segmentations, (255,255,255))
mask_inv = cv2.bitwise_not(mask)

masked_img = cv2.bitwise_and(img, mask)

background = np.ones_like(img, np.uint8)*255
background_masked = cv2.bitwise_and(background, mask_inv)

result = cv2.add(masked_img, background_masked)

#Convert BGR to RGB
result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

#cv2.imshow('target',result)

#Convert into a PIL (Pillow) image format before passing the extractor
reform_img = Image.fromarray(result)
colors = ext.extract_color(reform_img)
print(colors)

Ultralytics YOLOv8.0.28 🚀 Python-3.8.10 torch-1.13.1+cu116 CPU
YOLOv8s-seg summary (fused): 195 layers, 11779987 parameters, 0 gradients, 42.4 GFLOPs

0: 640x448 2 bottoms, 586.0ms
Speed: 5.0ms pre-process, 586.0ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 640)


['Beige', 'Grey']
